In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/amazon_reviews_us_Video_Games_v1_00.tsv', sep='\t', error_bad_lines=False)
df = pd.concat([df, pd.read_csv('data/amazon_reviews_us_Digital_Video_Games_v1_00.tsv', sep='\t', error_bad_lines=False)])
df = pd.concat([df, pd.read_csv('data/amazon_reviews_us_PC_v1_00.tsv', sep='\t', error_bad_lines=False)])
df.head()

b'Skipping line 20630: expected 15 fields, saw 22\nSkipping line 28172: expected 15 fields, saw 22\nSkipping line 54791: expected 15 fields, saw 22\n'
b'Skipping line 75419: expected 15 fields, saw 22\nSkipping line 104832: expected 15 fields, saw 22\n'
b'Skipping line 138464: expected 15 fields, saw 22\nSkipping line 194849: expected 15 fields, saw 22\n'
b'Skipping line 201568: expected 15 fields, saw 22\nSkipping line 242567: expected 15 fields, saw 22\n'
b'Skipping line 493585: expected 15 fields, saw 22\nSkipping line 502478: expected 15 fields, saw 22\n'
b'Skipping line 660750: expected 15 fields, saw 22\n'
b'Skipping line 42306: expected 15 fields, saw 22\nSkipping line 61136: expected 15 fields, saw 22\nSkipping line 64592: expected 15 fields, saw 22\n'
b'Skipping line 73268: expected 15 fields, saw 22\nSkipping line 80720: expected 15 fields, saw 22\nSkipping line 120624: expected 15 fields, saw 22\nSkipping line 121776: expected 15 fields, saw 22\n'
b'Skipping line 154351: exp

b'Skipping line 2978559: expected 15 fields, saw 22\nSkipping line 2998169: expected 15 fields, saw 22\nSkipping line 3003265: expected 15 fields, saw 22\nSkipping line 3003662: expected 15 fields, saw 22\n'
b'Skipping line 3035912: expected 15 fields, saw 22\nSkipping line 3054746: expected 15 fields, saw 22\n'
b'Skipping line 3301954: expected 15 fields, saw 22\n'
b'Skipping line 3563377: expected 15 fields, saw 22\n'
b'Skipping line 3952435: expected 15 fields, saw 22\n'
b'Skipping line 4144878: expected 15 fields, saw 22\n'
b'Skipping line 4208693: expected 15 fields, saw 22\nSkipping line 4217407: expected 15 fields, saw 22\n'
b'Skipping line 4382584: expected 15 fields, saw 22\n'
b'Skipping line 4794835: expected 15 fields, saw 22\nSkipping line 4840336: expected 15 fields, saw 22\n'
b'Skipping line 5002329: expected 15 fields, saw 22\nSkipping line 5012210: expected 15 fields, saw 22\nSkipping line 5021546: expected 15 fields, saw 22\n'
b'Skipping line 5117714: expected 15 field

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,12039526,RTIS3L2M1F5SM,B001CXYMFS,737716809,Thrustmaster T-Flight Hotas X Flight Stick,Video Games,5,0,0,N,Y,an amazing joystick. I especially love that yo...,"Used this for Elite Dangerous on my mac, an am...",2015-08-31
1,US,9636577,R1ZV7R40OLHKD,B00M920ND6,569686175,Tonsee 6 buttons Wireless Optical Silent Gamin...,Video Games,5,0,0,N,Y,Definitely a silent mouse... Not a single clic...,"Loved it, I didn't even realise it was a gami...",2015-08-31
2,US,2331478,R3BH071QLH8QMC,B0029CSOD2,98937668,Hidden Mysteries: Titanic Secrets of the Fatef...,Video Games,1,0,1,N,Y,One Star,poor quality work and not as it is advertised.,2015-08-31
3,US,52495923,R127K9NTSXA2YH,B00GOOSV98,23143350,GelTabz Performance Thumb Grips - PlayStation ...,Video Games,3,0,0,N,Y,"good, but could be bettee","nice, but tend to slip away from stick in inte...",2015-08-31
4,US,14533949,R32ZWUXDJPW27Q,B00Y074JOM,821342511,Zero Suit Samus amiibo - Japan Import (Super S...,Video Games,4,0,0,N,Y,Great but flawed.,"Great amiibo, great for collecting. Quality ma...",2015-08-31


In [3]:
cust_group = df.groupby('customer_id').filter(lambda x: x['review_id'].count() > 5)

In [4]:
max_review = cust_group.sort_values('review_date', ascending=False).drop_duplicates('customer_id')
max_review

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
4,US,14533949,R32ZWUXDJPW27Q,B00Y074JOM,821342511,Zero Suit Samus amiibo - Japan Import (Super S...,Video Games,4,0,0,N,Y,Great but flawed.,"Great amiibo, great for collecting. Quality ma...",2015-08-31
5155,US,17027466,R3GGY9K7SN20RY,B0052WT1OC,395540029,Flash Drive,PC,5,0,0,N,N,Great little USB,Have it loaded up with movies to take with me ...,2015-08-31
5210,US,14713147,RYMORIAXSSOJD,B010JGHL0Q,366925368,"Donop White USB Shock Wired Pc Controller,game...",PC,1,0,0,N,Y,Didn't Work For Me,While I don't like to give anything less than ...,2015-08-31
5203,US,13580771,R2WALFF1LN0EUS,B00QUMS068,234677384,SainSmart UNO R3 ATmega328P Development Board ...,PC,5,0,0,N,Y,Works great,Works great for the price. I have bough severa...,2015-08-31
5192,US,51429246,R3N1OKX6T6XSHK,B004EF5480,612421297,L-com DGB Series DB25 Female Connector for Fie...,PC,5,0,0,N,Y,Five Stars,excellent,2015-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1777663,US,50918923,REUTGBJC4JTAD,B000028U1X,728910037,Recoil - PC,Video Games,4,11,13,N,N,"Great game, few problems.",I think Recoil is a good game with lots of fun...,1999-12-05
1777732,US,50631006,R2PN7KRG4PFF19,B00002STPG,20688220,Suikoden,Video Games,4,2,3,N,N,A game that stays with you far after you've be...,"Now, I've played games since the beginning of ...",1999-12-04
1777794,US,50761966,R1RDH3QKAM9DP7,B00002STGT,942446495,Space Station: Silicon Valley,Video Games,5,9,10,N,N,Evo!,This game is great! It is one of a kind. Eve...,1999-12-04
1777853,US,50760484,R2TRRON972NICU,B00002STHN,567384800,NBA Courtside 2 featuring Kobe Bryant [Nintend...,Video Games,5,7,9,N,N,Grrrrreat,This is the best sim b-ball game I've played y...,1999-12-03


In [5]:
filtered_reviews = cust_group.drop(max_review.index)
filtered_reviews

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
24,US,22151364,R14Z48EC6DKJX9,B000PHNW3A,627665644,Shadowrun,Video Games,5,0,0,N,Y,Five Stars,Awesome!,2015-08-31
28,US,27975200,R3IU97WRKS5SLN,B0034JKYO6,718871603,Halo Wars,Video Games,5,0,0,N,Y,Five Stars,"great game, halo RTS well done",2015-08-31
34,US,10712640,R1DLJMMMK89CGD,B00KVRK39A,926940172,Persona Q: Shadow of the Labyrinth,Video Games,5,0,0,N,Y,Five Stars,A kickass special edition package for a great ...,2015-08-31
39,US,30067162,R1OTFNID62KK2T,B004UDLRMS,271964156,Resident Evil: Operation Raccoon City,Video Games,5,0,1,N,Y,Five Stars,great,2015-08-31
45,US,29604302,R9WP6WZ1ANYSD,B00O9GW8VK,949822044,Nintendo Super Smash Bros. Black Classic Gamec...,Video Games,5,0,0,N,Y,Look no further,a lot more comfortable and responsive than any...,2015-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6906822,US,50937664,R2HV10JP16SRHX,B00000J4L6,333716158,Netgear EN-108TP 10-Base T-Ethernet Hub 8-Port,PC,5,5,5,N,N,I'll get the job done. Thats all you need.,Pros: The 8th port is easy to configure for s...,1999-08-31
6906826,US,52233900,R1IYSGPKVIX1YL,B00000J4DT,321726475,Labtec C-324 Stereo Headset/Boom Microphone,PC,5,12,16,N,N,A great product,This is a great headset. It produces excellent...,1999-08-25
6906838,US,52751277,R1ME7QJFA2TRE5,B00000JBHA,865091807,Yamaha YSTMS28W Multimedia Speakers,PC,5,4,6,N,Y,Love these speakers,"These speakers sound great, and the little sat...",1999-08-13
6906848,US,50937664,R2A5CI6UOXPNHT,B00000JDKT,759728064,3dfx Voodoo3 2000 PCI 16MB Video Card,PC,5,3,6,N,N,Holy Smokes!,I can't say enough about this card! This is t...,1999-07-31


In [11]:
cust_group.to_csv('cust_group.csv')
max_review.to_csv('max_review.csv')
filtered_reviews.to_csv('filtered_reviews.csv')

In [10]:
print(len(cust_group))
print(len(max_review))
print(len(filtered_reviews))
print(len(max_review) + len(filtered_reviews))
print(len(cust_group) - (len(max_review) + len(filtered_reviews)))

2056995
193540
1826992
2020532
36463


In [7]:
print(len(cust_group.index))
print(len(max_review.index))
print(len(filtered_reviews.index))
print(len(max_review.index) + len(filtered_reviews.index))
print(len(cust_group.index) - (len(max_review.index) + len(filtered_reviews.index)))

2056995
193540
1826992
2020532
36463


In [8]:
missing = cust_group.drop(filtered_reviews.index).drop(max_review.index)
len(missing)

0

In [9]:
len(max_review)

193540

In [26]:
cluster_raw_max = max_review[ :len(max_review) // 2]
train_raw_max = max_review[len(max_review) // 2 : 7 * len(max_review) //  8]
test_raw_max = max_review[7 * len(max_review) // 8 : ]

indices_cluster = filtered_reviews['customer_id'].isin(cluster_raw_max['customer_id'])
indices_train = filtered_reviews['customer_id'].isin(train_raw_max['customer_id'])
cluster_raw = pd.concat([filtered_reviews[indices_cluster], cluster_raw_max])
train_raw = filtered_reviews[indices]
test_raw = filtered_reviews[~indices_cluster & ~indices_train]

print(len(train_raw_max))
print(len(test_raw_max))
print(len(cluster_raw))
print(len(train_raw))
print(len(test_raw))

72577
24193
1110213
418026
201365


In [28]:
cluster_raw_max.to_csv('cluster_raw_max.csv')
train_raw_max.to_csv('train_raw_max.csv')
test_raw_max.to_csv('test_raw_max.csv')
cluster_raw.to_csv('cluster_raw.csv')
train_raw.to_csv('train_raw.csv')
test_raw.to_csv('test_raw.csv')

In [29]:
filtered_reviews = pd.concat([filtered_reviews, cluster_raw_max])

In [33]:
def create_agg(df, col):
    criteria = {'review_id': 'count', 
            'star_rating': ['mean', 'std'], 
            'total_votes': ['mean', 'std'],
            'helpful_votes': ['mean', 'std']}
    return df.groupby(df[col]).agg(criteria).fillna(0)

In [41]:
product_df = create_agg(filtered_reviews, 'product_id')
cluster_df = create_agg(cluster_raw, 'customer_id')
train_cust_df = create_agg(train_raw, 'customer_id')
test_cust_df = create_agg(test_raw, 'customer_id')
train_lookup = train_raw_max[['customer_id', 'product_id', 'star_rating']]
test_lookup = test_raw_max[['customer_id', 'product_id', 'star_rating']]

In [42]:
train_df = pd.merge(train_lookup, train_cust_df, how='inner', on='customer_id')
train_df = pd.merge(train_df, product_df, how='inner', on='product_id')
train_df

,customer_id,product_id,star_rating_x,"('review_id', 'count')_x","('star_rating', 'mean')_x","('star_rating', 'std')_x","('total_votes', 'mean')_x","('total_votes', 'std')_x","('helpful_votes', 'mean')_x","('helpful_votes', 'std')_x","(review_id_y, count)","(star_rating_y, mean)","(star_rating_y, std)","(total_votes_y, mean)","(total_votes_y, std)","(helpful_votes_y, mean)","(helpful_votes_y, std)"
0,1617361,B001TH7GUA,5,5,5.000000,0.000000,0.200000,0.447214,0.000000,0.000000,231,4.883117,0.347939,0.506494,3.631828,0.445887,3.430727
1,53052607,B001TH7GUA,3,5,3.600000,0.547723,0.400000,0.894427,0.400000,0.894427,231,4.883117,0.347939,0.506494,3.631828,0.445887,3.430727
2,16367779,B001TH7GUA,5,9,4.444444,0.527046,0.888889,1.364225,0.777778,1.092906,231,4.883117,0.347939,0.506494,3.631828,0.445887,3.430727
3,25485198,B001TH7GUA,5,17,5.000000,0.000000,0.235294,0.437237,0.176471,0.392953,231,4.883117,0.347939,0.506494,3.631828,0.445887,3.430727
4,52598487,B001TH7GUA,5,9,3.222222,1.201850,3.333333,4.690416,2.888889,4.755114,231,4.883117,0.347939,0.506494,3.631828,0.445887,3.430727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45388,47544998,B00GG69IJQ,2,5,4.000000,1.732051,7.400000,14.328294,7.000000,13.435029,6,4.166667,0.408248,0.833333,1.602082,0.833333,1.602082
45389,17514911,B003D18Q7W,5,7,4.285714,1.253566,0.857143,1.214986,0.714286,1.112697,59,4.372881,1.244274,0.254237,0.957605,0.237288,0.837463
45390,34290997,B0030B6F1O,3,5,5.000000,0.000000,1.000000,1.414214,0.800000,1.095445,54,3.462963,1.513623,0.129630,0.615715,0.111111,0.571878
45391,20758058,B00FAC0ZYY,5,9,5.000000,0.000000,0.777778,1.715938,0.444444,1.013794,21,4.857143,0.358569,0.571429,0.810643,0.142857,0.478091


In [44]:
test_df = pd.merge(test_lookup, test_cust_df, how='inner', on='customer_id')
test_df = pd.merge(test_df, product_df, how='inner', on='product_id')
test_df

,customer_id,product_id,star_rating_x,"('review_id', 'count')_x","('star_rating', 'mean')_x","('star_rating', 'std')_x","('total_votes', 'mean')_x","('total_votes', 'std')_x","('helpful_votes', 'mean')_x","('helpful_votes', 'std')_x","(review_id_y, count)","(star_rating_y, mean)","(star_rating_y, std)","(total_votes_y, mean)","(total_votes_y, std)","(helpful_votes_y, mean)","(helpful_votes_y, std)"
0,36330222,B00B829UQY,5,24,4.291667,1.267629,0.541667,1.250362,0.333333,1.049500,19,4.368421,1.164785,0.789474,2.507299,0.684211,2.286497
1,24360083,B00B829UQY,4,23,4.347826,1.191206,1.434783,2.191431,1.000000,1.537412,19,4.368421,1.164785,0.789474,2.507299,0.684211,2.286497
2,28891040,B005KDYBIO,4,10,4.500000,0.707107,0.000000,0.000000,0.000000,0.000000,241,4.485477,0.935858,15.925311,136.525923,13.975104,122.943955
3,52449052,B005KDYBIO,3,5,3.400000,1.516575,11.800000,20.789420,10.400000,18.187908,241,4.485477,0.935858,15.925311,136.525923,13.975104,122.943955
4,27192976,B005KDYBIO,4,6,4.666667,0.816497,0.500000,1.224745,0.000000,0.000000,241,4.485477,0.935858,15.925311,136.525923,13.975104,122.943955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22924,50918923,B000028U1X,4,5,4.400000,0.894427,15.400000,17.357995,8.600000,7.021396,2,3.500000,0.707107,2.000000,2.828427,2.000000,2.828427
22925,50631006,B00002STPG,4,5,4.000000,0.707107,12.600000,16.994117,10.400000,14.117365,14,4.357143,0.928783,3.857143,6.803361,1.571429,1.988981
22926,50761966,B00002STGT,5,5,4.400000,0.894427,11.000000,2.449490,8.400000,3.209361,8,4.875000,0.353553,1.125000,2.416461,1.000000,2.070197
22927,50760484,B00002STHN,5,6,3.666667,1.211060,5.333333,3.011091,2.833333,2.228602,8,4.375000,1.060660,1.125000,2.031010,0.625000,0.744024


In [45]:
train_df.to_csv('train_df.csv')
test_df.to_csv('test_df.csv')

In [ ]:
# Barrier between old and new code

In [110]:
criteria = {'review_id': 'count', 
            'star_rating': ['mean', 'std'], 
            'total_votes': ['mean', 'std'],
            'helpful_votes': ['mean', 'std']}
customer_df = train_df.groupby(train_df.customer_id).agg(criteria).fillna(0)
product_df = train_df.groupby(train_df.product_id).agg(criteria).fillna(0)
lookup_table = train_df[['customer_id', 'product_id', 'star_rating']]

,customer_id,product_id,star_rating_x,"('review_id', 'count')_x","('star_rating', 'mean')_x","('star_rating', 'std')_x","('total_votes', 'mean')_x","('total_votes', 'std')_x","('helpful_votes', 'mean')_x","('helpful_votes', 'std')_x","(review_id_y, count)","(star_rating_y, mean)","(star_rating_y, std)","(total_votes_y, mean)","(total_votes_y, std)","(helpful_votes_y, mean)","(helpful_votes_y, std)"
0,14000174,B002GHHJRC,2,1,2.0,0.000000,38.0,0.0,37.0,0.0,1,2.0,0.000000,38.0,0.0,37.0,0.0
1,28853700,B001IKD5FS,2,1,2.0,0.000000,0.0,0.0,0.0,0.0,1,2.0,0.000000,0.0,0.0,0.0,0.0
2,33673543,B002PAW5PQ,5,1,5.0,0.000000,0.0,0.0,0.0,0.0,1,5.0,0.000000,0.0,0.0,0.0,0.0
3,35130971,B001PKHRUK,5,2,4.5,0.707107,0.0,0.0,0.0,0.0,2,4.5,0.707107,0.0,0.0,0.0,0.0
4,35130971,B001UU1WN8,4,2,4.5,0.707107,0.0,0.0,0.0,0.0,2,4.5,0.707107,0.0,0.0,0.0,0.0


In [16]:
import numpy as np
swapped = np.swapaxes(customer_df,0,1)
swapped.shape

(7, 1125953)

In [34]:
import skfuzzy as fuzz

In [36]:
models = []
for num_clusters in range(2, 8):
    print('\n---', num_clusters, '---')
    cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(swapped, num_clusters, 2, 0.005, 1000)
    models.append((cntr, u, u0, d, jm, p, fpc))
    print('FPC:', fpc)
    cluster_membership = np.argmax(u, axis=0)
    for i  in range(num_clusters):
        cluster = cluster_membership[cluster_membership == i]
        print('Cluster {}: {}%'.format(i, (len(cluster) / len(cluster_membership)) * 100))


--- 2 ---
FPC: 0.9661281718166361
Cluster 0: 2.2099501488960906%
Cluster 1: 97.79004985110392%

--- 3 ---
FPC: 0.9419680282920196
Cluster 0: 4.651437493394484%
Cluster 1: 95.2986492331385%
Cluster 2: 0.049913273467009724%

--- 4 ---
FPC: 0.8996314574719949
Cluster 0: 90.6400178337817%
Cluster 1: 0.6568657839181565%
Cluster 2: 0.0164305259633395%
Cluster 3: 8.68668585633681%

--- 5 ---
FPC: 0.8725045217103989
Cluster 0: 0.0036413598080914564%
Cluster 1: 0.06589973116106977%
Cluster 2: 87.77977411135278%
Cluster 3: 10.79378979406778%
Cluster 4: 1.3568950036102752%

--- 6 ---
FPC: 0.8233155189158615
Cluster 0: 13.927490756718974%
Cluster 1: 2.9347583780140023%
Cluster 2: 0.026288841541343196%
Cluster 3: 82.765532841957%
Cluster 4: 0.3448634179224177%
Cluster 5: 0.0010657638462706702%

--- 7 ---
FPC: 0.774704689656206
Cluster 0: 0.8883141658666037%
Cluster 1: 0.10524417981922869%
Cluster 2: 0.0008881365385588919%
Cluster 3: 78.09224718971396%
Cluster 4: 4.56058112549991%
Cluster 5: 0.0115

In [41]:
cluster_pcts = np.swapaxes(models[5][1],0,1)
cluster_pcts.shape

(1125953, 7)

In [53]:
cluster_df = pd.DataFrame(cluster_pcts, index=customer_df.index, columns=['cluster_0', 
                                                                          'cluster_1', 
                                                                          'cluster_2',
                                                                          'cluster_3',
                                                                          'cluster_4', 
                                                                          'cluster_5',
                                                                          'cluster_6'])
cluster_df.head()

,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6
customer_id,,,,,,,
10018,0.001394,0.000108,3.086527e-07,0.822362,0.012592,0.000008,0.163536
10114,0.000278,0.000022,6.394714e-08,0.973026,0.002339,0.000002,0.024332
10146,0.000278,0.000022,6.147077e-08,0.964092,0.002517,0.000002,0.033090
10164,0.000245,0.000020,5.627820e-08,0.976589,0.002056,0.000001,0.021089
10192,0.000245,0.000020,5.627820e-08,0.976589,0.002056,0.000001,0.021089


In [54]:
customer_combined_df = pd.concat([customer_df, cluster_df], axis=1)
customer_combined_df

,"(review_id, count)","(star_rating, mean)","(star_rating, std)","(total_votes, mean)","(total_votes, std)","(helpful_votes, mean)","(helpful_votes, std)",cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6
customer_id,,,,,,,,,,,,,,
10018,4,4.25,0.500000,1.25,0.957427,0.25,0.500000,0.001394,0.000108,3.086527e-07,0.822362,0.012592,0.000008,0.163536
10114,2,5.00,0.000000,0.00,0.000000,0.00,0.000000,0.000278,0.000022,6.394714e-08,0.973026,0.002339,0.000002,0.024332
10146,1,5.00,0.000000,1.00,0.000000,1.00,0.000000,0.000278,0.000022,6.147077e-08,0.964092,0.002517,0.000002,0.033090
10164,1,5.00,0.000000,0.00,0.000000,0.00,0.000000,0.000245,0.000020,5.627820e-08,0.976589,0.002056,0.000001,0.021089
10192,1,5.00,0.000000,0.00,0.000000,0.00,0.000000,0.000245,0.000020,5.627820e-08,0.976589,0.002056,0.000001,0.021089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53096494,2,2.50,2.121320,14.00,7.071068,12.50,7.778175,0.022167,0.001107,2.726586e-06,0.130024,0.593656,0.000069,0.252973
53096501,1,5.00,0.000000,0.00,0.000000,0.00,0.000000,0.000245,0.000020,5.627820e-08,0.976589,0.002056,0.000001,0.021089
53096538,2,3.00,2.828427,2.50,2.121320,2.00,1.414214,0.002422,0.000180,5.037979e-07,0.505556,0.024798,0.000012,0.467031


In [71]:
len(df.query('review_date >= "2014-12-20"')) / len(df)

0.20105641997473236